In [1]:
from __future__ import absolute_import, division, print_function

import os
import tensorflow as tf
from tensorflow_transform.tf_metadata import dataset_metadata, metadata_io,dataset_schema 
from tensorflow_transform.beam.tft_beam_io import transform_fn_io
tf.enable_eager_execution()

In [2]:
os.getcwd()

'/Users/Niklas/Documents/Named_entity_recognition/model/notebooks'

# HAVE TO RE RUN ALL! AND CLEAR FOLDERS

https://guillaumegenthial.github.io/sequence-tagging-with-tensorflow.html

In [3]:
transformed_metadata = metadata_io.read_metadata(
    os.path.join(
      "gs://named_entity_recognition/beam/", transform_fn_io.TRANSFORMED_METADATA_DIR
      )
    )
transformed_feature_spec = transformed_metadata.schema.as_feature_spec()

In [4]:
transformed_feature_spec

{u'chars': FixedLenFeature(shape=[30, 10], dtype=tf.int64, default_value=None),
 u'chars_in_word': FixedLenFeature(shape=[30], dtype=tf.int64, default_value=None),
 u'id': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None),
 u'label': FixedLenFeature(shape=[], dtype=tf.string, default_value=None),
 u'label_length': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None),
 u'labels': FixedLenFeature(shape=[30], dtype=tf.int64, default_value=None),
 u'sentence_length': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None),
 u'text': FixedLenFeature(shape=[], dtype=tf.string, default_value=None),
 u'word_representation': VarLenFeature(dtype=tf.int64)}

In [5]:
def _read_from_tfrecord(example_proto):
    features = tf.parse_example([example_proto], features=transformed_feature_spec)

    return features

In [6]:
output_list=[]

In [7]:
ds = tf.data.TFRecordDataset("gs://named_entity_recognition/beam/TEST-00000-of-00005").map(_read_from_tfrecord).batch(2).make_one_shot_iterator()
for x in ds:
    output_list.append(x)


In [8]:
count=0
for x in output_list:
    print(x["sentence_length"])
    print(x["text"])
    count=count +1
    if count>2:
        break

tf.Tensor(
[[9]
 [3]], shape=(2, 1), dtype=int64)
tf.Tensor(
[['CRICKET - ENGLAND V PAKISTAN FINAL TEST SCOREBOARD .']
 ['BUENOS AIRES 1996-08-26']], shape=(2, 1), dtype=string)
tf.Tensor(
[[6]
 [4]], shape=(2, 1), dtype=int64)
tf.Tensor(
[['Fastest lap : Aoki 147.786 kph']
 ['-- Chicago newsdesk 312-408-8787']], shape=(2, 1), dtype=string)
tf.Tensor(
[[35]
 [ 9]], shape=(2, 1), dtype=int64)
tf.Tensor(
[['Most including former president F.W. de Klerk and African National Congress Deputy President Thabo Mbeki offered apologies for any mistakes they had made and accepted broad responsibility for the actions of their foot soldiers .']
 ['No1 Bare Bright 91 to 92 cents / pound']], shape=(2, 1), dtype=string)


In [9]:
from string import ascii_lowercase
MAPPING = {a:index for index,a in enumerate(ascii_lowercase + ascii_lowercase.upper())}

In [13]:
fruit = 'Apple'
isApple = True if fruit == 'Apple' else False

a = 3
b = a if a<5 else 5
b

3

In [10]:
x.keys()

[u'word_representation',
 u'text',
 u'chars',
 u'labels',
 u'label',
 u'label_length',
 u'sentence_length',
 u'id',
 u'chars_in_word']

In [11]:
print(x['labels'])
print(x['sentence_length'])
print(tf.reshape(x['sentence_length'],[2]))

tf.Tensor(
[[[8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]]

 [[8 8 6 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]]], shape=(2, 1, 30), dtype=int64)
tf.Tensor(
[[15]
 [19]], shape=(2, 1), dtype=int64)
tf.Tensor([15 19], shape=(2,), dtype=int64)


In [12]:
b = x["text"].numpy()
b

array([['I got the good part of the bat on it and it carried out .'],
       ['The 36-year-old Briton is still considering the offer and is expected to announce his decision later on Wednesday .']],
      dtype=object)

In [13]:
print(x['sentence_length'])

tf.Tensor(
[[15]
 [19]], shape=(2, 1), dtype=int64)


In [14]:
SENTENCE_LEN=30
WORD_LEN=10
BATCH_SIZE=2
CHAR_SIZE=54
EMBED_DIM_CHAR=int(CHAR_SIZE**(1/4))*8
DEFAULT_WORD_VALUE =0
VOCAB_SIZE=1e4
EMBED_DIM_WORD=int(VOCAB_SIZE**(1/4))*8
input_char = tf.reshape(x["chars"],[BATCH_SIZE,SENTENCE_LEN,WORD_LEN])
char_embedding = tf.contrib.layers.embed_sequence(
    input_char, vocab_size=CHAR_SIZE, embed_dim=EMBED_DIM_CHAR)

In [15]:
# No padding needed now :) 
input_word = tf.sparse_to_dense(x["word_representation"].indices,[BATCH_SIZE,1,SENTENCE_LEN],
                           x["word_representation"].values,default_value=DEFAULT_WORD_VALUE
                          )
input_word = tf.reshape(input_word,[BATCH_SIZE,SENTENCE_LEN])
word_embedding = tf.contrib.layers.embed_sequence(
    input_word, vocab_size=VOCAB_SIZE, embed_dim=EMBED_DIM_WORD)

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


In [16]:
x["labels"]

<tf.Tensor: id=86, shape=(2, 1, 30), dtype=int64, numpy=
array([[[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
         8, 8, 8, 8, 8, 8, 8, 8, 8]],

       [[8, 8, 6, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
         8, 8, 8, 8, 8, 8, 8, 8, 8]]])>

In [17]:
x["labels"]

<tf.Tensor: id=86, shape=(2, 1, 30), dtype=int64, numpy=
array([[[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
         8, 8, 8, 8, 8, 8, 8, 8, 8]],

       [[8, 8, 6, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
         8, 8, 8, 8, 8, 8, 8, 8, 8]]])>

In [18]:
input_word

<tf.Tensor: id=29845, shape=(2, 30), dtype=int64, numpy=
array([[  59,  803,    1,  392,  260,    2,    1, 1622,    9,   33,    8,
          33, 2613,   64,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0],
       [  15,    0, 4812,   26,  239, 2346,    1, 1012,    8,   26,  176,
           4, 2370,   34,  534,  492,    9,   71,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]])>

In [19]:
# Print the shapes 
print(tf.shape(input_word))
print(tf.shape(word_embedding))
print(tf.shape(char_embedding))

tf.Tensor([ 2 30], shape=(2,), dtype=int32)
tf.Tensor([ 2 30 80], shape=(3,), dtype=int32)
tf.Tensor([ 2 30 10 16], shape=(4,), dtype=int32)


In [20]:
s = tf.shape(char_embedding)
char_embeddings = tf.reshape(char_embedding, shape=[-1, s[-2], s[-1]])
word_lengths = tf.reshape(x['chars_in_word'],[-1])

In [21]:
CHAR_HIDDEN_SIZE=10
# 3. bi lstm on chars
cell_fw = tf.contrib.rnn.LSTMCell(CHAR_HIDDEN_SIZE, state_is_tuple=True)
cell_bw = tf.contrib.rnn.LSTMCell(CHAR_HIDDEN_SIZE, state_is_tuple=True)

_, ((_, output_fw), (_, output_bw)) = tf.nn.bidirectional_dynamic_rnn(cell_fw,
    cell_bw, char_embeddings, sequence_length=word_lengths,
    dtype=tf.float32)
#shape = (batch x sentence, 2 x char_hidden_size)
output = tf.concat([output_fw, output_bw], axis=-1)

# shape = (batch, sentence, 2 x char_hidden_size)
char_rep = tf.reshape(output, shape=[-1, s[1], 2*CHAR_HIDDEN_SIZE])

# shape = (batch, sentence, 2 x char_hidden_size + word_vector_size)
word_embeddings = tf.concat([word_embedding, char_rep], axis=-1)

In [22]:
word_lengths

<tf.Tensor: id=29883, shape=(60,), dtype=int64, numpy=
array([ 1,  3,  3,  4,  4,  2,  3,  3,  2,  2,  3,  2,  7,  3,  1,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3, 10,  6,  2,
        5, 10,  3,  5,  3,  2,  8,  2,  8,  3,  8,  5,  2,  9,  1,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0])>

In [23]:
dim_words = tf.shape(char_embeddings)[1]
dim_chars = tf.shape(char_embeddings)[2]
flat = tf.reshape(char_embeddings, [-1, dim_words,EMBED_DIM_CHAR])


In [24]:
tf.shape(char_embeddings)
t = tf.transpose(char_embeddings, perm=[1, 0, 2])
print(t)

tf.Tensor(
[[[-0.276442    0.09469041 -0.24847177 ...  0.23459393 -0.0731729
    0.15602005]
  [ 0.29159194 -0.22921945 -0.25485438 ...  0.14288145  0.2496165
   -0.10974234]
  [ 0.2593913  -0.24994501 -0.00471407 ...  0.19682276 -0.271191
   -0.17006284]
  ...
  [-0.276442    0.09469041 -0.24847177 ...  0.23459393 -0.0731729
    0.15602005]
  [-0.276442    0.09469041 -0.24847177 ...  0.23459393 -0.0731729
    0.15602005]
  [-0.276442    0.09469041 -0.24847177 ...  0.23459393 -0.0731729
    0.15602005]]

 [[-0.276442    0.09469041 -0.24847177 ...  0.23459393 -0.0731729
    0.15602005]
  [-0.276442    0.09469041 -0.24847177 ...  0.23459393 -0.0731729
    0.15602005]
  [ 0.23106164 -0.02746925 -0.13947603 ...  0.22402978  0.12154296
   -0.27353936]
  ...
  [-0.276442    0.09469041 -0.24847177 ...  0.23459393 -0.0731729
    0.15602005]
  [-0.276442    0.09469041 -0.24847177 ...  0.23459393 -0.0731729
    0.15602005]
  [-0.276442    0.09469041 -0.24847177 ...  0.23459393 -0.0731729
    0.1

In [25]:
print(tf.shape(char_embeddings))
print(dim_chars)
print(dim_words)
print(EMBED_DIM_CHAR)

tf.Tensor([60 10 16], shape=(3,), dtype=int32)
tf.Tensor(16, shape=(), dtype=int32)
tf.Tensor(10, shape=(), dtype=int32)
16


In [26]:
# WORD LSTM:S

HIDDEN_SIZE=20
cell_fw = tf.contrib.rnn.LSTMCell(HIDDEN_SIZE)
cell_bw = tf.contrib.rnn.LSTMCell(HIDDEN_SIZE)
sequence_length= tf.reshape(x['sentence_length'],[BATCH_SIZE])
(output_fw, output_bw), _ = tf.nn.bidirectional_dynamic_rnn(cell_fw,
    cell_bw, word_embeddings, sequence_length=sequence_length,
    dtype=tf.float32)
context_rep = tf.concat([output_fw, output_bw], axis=-1)

In [53]:
sequence_length

<tf.Tensor: id=30916, shape=(2,), dtype=int64, numpy=array([15, 19])>

In [52]:
word_embeddings

<tf.Tensor: id=30891, shape=(2, 30, 100), dtype=float32, numpy=
array([[[-0.01792703, -0.00522212,  0.01281289, ...,  0.02264906,
          0.00806679, -0.06072659],
        [ 0.00194441, -0.01352117,  0.02402751, ...,  0.05622904,
          0.00528315, -0.07553377],
        [ 0.01745144,  0.02065966, -0.01256571, ..., -0.03619995,
         -0.01457127,  0.04420982],
        ...,
        [ 0.01814665, -0.00356887,  0.00626756, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.01814665, -0.00356887,  0.00626756, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.01814665, -0.00356887,  0.00626756, ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.00730885, -0.001112  , -0.01668721, ...,  0.05332617,
          0.03637786, -0.12949157],
        [ 0.01814665, -0.00356887,  0.00626756, ...,  0.0767253 ,
          0.07335322, -0.04356287],
        [-0.01765737,  0.02409077, -0.00847279, ..., -0.00430214,
          0.08887688, -0.05714823],
   

In [46]:
tf.shape(sequence_length)
sequence_length.numpy()

array([15, 19])

In [43]:
tf.shape(word_embeddings)

<tf.Tensor: id=36699, shape=(3,), dtype=int32, numpy=array([  2,  30, 100], dtype=int32)>

In [44]:
tf.shape(context_rep)

<tf.Tensor: id=36701, shape=(3,), dtype=int32, numpy=array([ 2, 30, 40], dtype=int32)>

In [28]:
CLASSES_OUTPUT=9

ntime_steps = tf.shape(context_rep)[1]
context_rep_flat = tf.reshape(context_rep, [-1, 2*HIDDEN_SIZE])
preds = tf.layers.dense(inputs = context_rep_flat,units=CLASSES_OUTPUT)
logits = tf.reshape(preds, [-1, ntime_steps, CLASSES_OUTPUT])


In [29]:
crf_params = tf.get_variable("crf", [CLASSES_OUTPUT, CLASSES_OUTPUT], dtype=tf.float32)
pred_ids, _ = tf.contrib.crf.crf_decode(logits, crf_params, sequence_length)

In [30]:
sequence_length

<tf.Tensor: id=30916, shape=(2,), dtype=int64, numpy=array([15, 19])>

In [31]:
sequence_length

<tf.Tensor: id=30916, shape=(2,), dtype=int64, numpy=array([15, 19])>

In [32]:
weights = tf.sequence_mask(sequence_length,maxlen=20)
weights
#tf.metrics.accuracy(labels, pred_ids, weights)

<tf.Tensor: id=35453, shape=(2, 20), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True, False, False, False,
        False, False],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True, False]])>

In [33]:
pred_ids

<tf.Tensor: id=35441, shape=(2, 30), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>

In [34]:

labels = tf.reshape(x['labels'],[BATCH_SIZE,SENTENCE_LEN])
labels

<tf.Tensor: id=35457, shape=(2, 30), dtype=int64, numpy=
array([[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8],
       [8, 8, 6, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8]])>

In [35]:
logits

<tf.Tensor: id=33635, shape=(2, 30, 9), dtype=float32, numpy=
array([[[ 1.00542437e-02, -5.97223081e-03,  2.55422899e-03,
          1.58257037e-02,  2.99501233e-03, -1.23573449e-02,
          1.14818653e-02, -1.48824723e-02,  1.87883638e-02],
        [ 1.41725074e-02, -2.30121613e-03,  5.74503094e-03,
          1.55944154e-02,  1.78433256e-04, -8.53263028e-03,
          1.14082890e-02, -1.93446446e-02,  1.97767559e-02],
        [ 2.16319989e-02,  7.90296588e-04,  1.70470439e-02,
          1.85925495e-02, -1.06485840e-02, -1.27577577e-02,
          5.96372504e-03, -1.14536602e-02,  5.58378175e-03],
        [ 1.23999678e-02, -3.46580031e-03,  7.40117300e-03,
          1.20074339e-02, -8.95404816e-03, -3.34595423e-03,
          8.69130343e-03, -7.45484699e-03,  4.09019180e-03],
        [ 1.67719908e-02,  1.93982502e-03,  1.98841188e-02,
          1.72285754e-02, -2.10856088e-02,  7.34140538e-03,
          3.26080993e-03, -7.19738146e-03,  8.64688307e-04],
        [ 1.44431170e-03, -1.6867

In [36]:
log_likelihood, _ = tf.contrib.crf.crf_log_likelihood(
        logits, labels, sequence_length, crf_params)
loss = tf.reduce_mean(-log_likelihood)

In [37]:
loss

<tf.Tensor: id=36686, shape=(), dtype=float32, numpy=45.423134>

In [ ]:
LOOK_UP_TABLE="FILE_NAME_TO_LOOP_UP_VOCAB"

if mode == tf.estimator.ModeKeys.PREDICT:
    # Predictions
    # GET THE WORD FORM THE ID, LINK THIS WITH THE BEAM TRANSFORM VOCAB
    reverse_vocab_tags = tf.contrib.lookup.index_to_string_table_from_file(
        LOOK_UP_TABLE)
    pred_strings = reverse_vocab_tags.lookup(tf.to_int64(pred_ids))
    predictions = {
        'pred_ids': pred_ids,
        'tags': pred_strings
    }
    return tf.estimator.EstimatorSpec(mode, predictions=predictions)
else:
    # Loss
    tags = labes
    log_likelihood, _ = tf.contrib.crf.crf_log_likelihood(
        logits, tags, nwords, crf_params)
    loss = tf.reduce_mean(-log_likelihood)

    # Metrics
    # AWESOME THIS IS GREAT TO USE THIS MASK HERE
    # WILL SOLVE A LOT OF PROBLEMS, OTHERWISE
    # GOOD LEARNING AS WELL
    weights = tf.sequence_mask(nwords)
    metrics = {
        'acc': tf.metrics.accuracy(tags, pred_ids, weights),
        'precision': precision(tags, pred_ids, num_tags, indices, weights),
        'recall': recall(tags, pred_ids, num_tags, indices, weights),
        'f1': f1(tags, pred_ids, num_tags, indices, weights),
    }
    # ADD THEM ALL TO THE GRAPH
    for metric_name, op in metrics.items():
        tf.summary.scalar(metric_name, op[1])
    
    # WHAT WILL HAPPEN IF WE HAVE EVAL MODE
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode, loss=loss, eval_metric_ops=metrics)

    # WHAT WILL HAPPEN IF WE HAVE TRAIN? 
    elif mode == tf.estimator.ModeKeys.TRAIN:
        train_op = tf.train.AdamOptimizer().minimize(
            loss, global_step=tf.train.get_or_create_global_step())
        return tf.estimator.EstimatorSpec(
            mode, loss=loss, train_op=train_op)

In [ ]:
NUM_CLASSES = 9
NUM_CLASSES = tf.constant(NUM_CLASSES)
NUM_CLASSES